## Required Libraries

In [1]:
import pandas as pd
import numpy as np
from urllib.request import urlopen, Request
import re
import requests
from bs4 import BeautifulSoup

## Given Url

In [4]:
url = r"https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_1"

## Data Ectraction

In [5]:
req = Request(url)
rawpage = urlopen(req).read().decode()

In [6]:
soup = BeautifulSoup(rawpage, 'html.parser')

In [8]:
div_list = soup.find_all('div', {"data-component-type": "s-search-result"})

In [13]:
URL_list = []
Name_list = []
Product_Price = []
Rating_list = []
Reviews_number_list = []
asin_list = []

def data_extraction(data):
    base_url = data.h2.a['href']

    final_url = 'https://www.amazon.in' + base_url 
    URL_list.append(final_url)
    
    name = data.find('span', {'class':"a-size-medium a-color-base a-text-normal"}).text
    Name_list.append(name)
    
    price = data.find('span',{'class':"a-price-whole"}).text
    price = int(re.sub('[,()]', '',price))
    Product_Price.append(price)
    
    rating = data.find('span',{'class':"a-icon-alt" }).text
    rating = float(rating[:3])
    Rating_list.append(rating)

    review = data.find('span',{'class' : "a-size-base s-underline-text" }).text
    review = int(re.sub('[,()]', '',review))
    Reviews_number_list.append(review)
    
    x = str(data)
    asin =  re.findall('data-asin=(.*?) data-component-type',x)[0][1:-1]
    asin_list.append(asin)


#     print(final_url,'\n',name,'\n',price,'\n',review,'\n',rating)

In [14]:
for i in div_list:
    data_extraction(i)

## Create links to go on multiple pages

In [15]:
link_list = []
for i in range(2,25):
    link = f'https://www.amazon.in/s?k=bags&page=4&crid=2M096C61O4MLT&qid=1675400625&sprefix=ba%2Caps%2C283&ref=sr_pg_{i}'
    link_list.append(link)

## Data Extractin from multiple pages

In [38]:
soup_list = []
for i in link_list:
    print(i)
    req = Request(i)
    rawpage = urlopen(req).read().decode()

    soup = BeautifulSoup(htmlContent, 'html.parser')
    soup_list.append(soup)

In [31]:
list_of_div_list = []
for i in soup_list:
    div_list = i.find_all('div', {"data-component-type": "s-search-result"})
    list_of_div_list.append(div_list)

In [33]:
for i in list_of_div_list:
    for j in i:
        data_extraction(j)

## Creating DataFrame

In [40]:
df = pd.DataFrame({'Product_URL' : URL_list,'Product_Name' : Name_list,  'Product_Price' : Product_Price,
                   'Rating_list' : Rating_list ,'Number_of_reviews' : Reviews_number_list, 'ASIN' : asin_list})

## Save to CSV file

In [42]:
df.to_csv('Output.csv')